# Processing Visium Data to visualize in cellxgene

In [ ]:
import scanpy as sc
import pandas as pd

### Read the file by `read_visium` function

You need to indicate the **folder path** that includes `spatial` folder and `filtered_feature_bc_matrix.h5` file. This path is usually the output path of running [SpaceRanger](https://support.10xgenomics.com/spatial-gene-expression/software/pipelines/latest/what-is-space-ranger) on your data.

In [ ]:
adata = sc.read_visium('/path/to/spaceranger/outs')


### Read the cell metadata by `read_csv` function

Data type needs to be changed with `.astype('category')` if you want them to be shown as a as collapsible items on the left hand side of cellxgene. This should only be applied to metadata that **is not** continuous.

In [ ]:
adata.obsm['X_umap'] = pd.read_csv('/path/to/analysis/umap/projection.csv', usecols = ['UMAP-1', 'UMAP-2']).to_numpy()
adata.obsm['X_tsne'] = pd.read_csv('/path/to/analysis/tsne/projection.csv', usecols = ['TSNE-1', 'TSNE-2']).to_numpy()
adata.obs['clustering'] = pd.read_csv('/path/to/clustering/clusters.csv', index_col = 'Barcode')
adata.obs['clustering'] = adata.obs['clustering'].astype('category')


### Rename `spatial` as `X_spatial`

In [ ]:
adata.obsm['X_spatial'] = adata.obsm['spatial']
del adata.obsm['spatial']

cellxgene only reads multi-dimensional annotation of observations (`obsm`) embedeings if they are prefixed with `X_` 
that's why we need to rename `adata.obsm['spatial']` to `adata.obsm['X_spatial']`.
Other embedings that where already prefixed are: `X_umap` and `X_tsne`.

#### Or transfer spatial features from another h5ad
Only do this if your spatial features come from another H5AD object

In [ ]:
adata2 = sc.read('/path/to/other/object.h5ad')
adata.uns['spatial'] = adata2.uns['spatial']
adata.obsm['X_spatial'] = adata2.obsm['X_spatial']


### Integrate Cell2Location output to your AnnData

In [ ]:
# if your cell abundance data are in csv files
cell_abundance = pd.read_csv('/your/path/to/cell_abundance.csv')
adata.obs = pd.concat([adata.obs, cell_abundance], axis=1)
adata.obsm['q05_cell_abundance_w_sf'] = pd.read_csv('/your/path/to/csv')

# if your cell abundance data are in obsm slot
adata.obs = pd.concat([adata.obs, adata.obsm['q05_cell_abundance_w_sf']], axis=1)

Cell abundance values are continuous so they won't be shown as categories but rather as histograms on the left side of cellxgene as long as they are part of the one-dimensional annotation of observations (`adata.obs`).

### Save your AnnData

In [ ]:
adata.write('/path/for/save/visium.h5ad', compression = 'gzip')

Compressing the output file will make it use less storage when on disk, but it may make the reading-writing of the file take longer. Use this if you're tight on storage or need to share files over internet and copying/uploading takes too long.

---

# Multiple Visium samples for cellxgene (mosaic)

If you want to add multiple samples into a single Anndata, you need to follow these steps:

### Read all visium samples using scanpy.read_visium

In [ ]:
samples = [ "LibID_1", "LibID_2","LibID_3" ]

This will assume that all your space ranger outputs are inside the folder `samples` and that each one is on it's separate folder matching the sample name (or the name that you've filled the `samples` array with).

Each sub-folder must include at least the `filtered_feature_bc_matrix.h5` file and a `spatial` folder with `scalefactors_json.json`, `tissue_hires_image.png` and `tissue_positions_list.csv`.

In [ ]:
adatas = []
spatial_uns = {} # use this to keep the unstructured observations when concatenating
for sample in samples:
    print(f"- Reading {sample}")
    adata = sc.read_visium(f"samples/{sample}")
    adata.var_names_make_unique()
    # create a new observation to identify each library id
    adata.obs['library_id'] = sample
    # also prepend library id to barcodes (libID@Barcode) to prevent observation names not being unique
    adata.obs.set_index(sample + "@" + adata.obs.index.astype(str), inplace=True)
    spatial_uns[sample] = adata.uns["spatial"][sample]
    adata.obs['library_id'].astype("category")
    adatas.append(adata)

### Merge objects into a single Anndata (h5ad)

In [ ]:
adata = sc.concat(adatas)
adata.uns['spatial'] = spatial_uns

### Create a single image with all Visium slides from spaceranger outs

In [ ]:
import math
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

sample_count = len(samples)

# set the max item count for each dimensions (row/count)
N = math.ceil(math.sqrt(sample_count))
print(f"max {N} images per row/column")

# because the max dimension of a hires image is 2000px
max_width = 2000
max_height = 2000

# estimate final canvas size based on 2000x2000 max image size
canvas = Image.new('RGBA', (max_width*N, max_height*N))

# paste all images in the canvas
# fill row by row taking N samples at the time
y = 0
for i in range(0, sample_count, N):
    x = 0
    for sample in samples[i:i+N]:
        # add this sample's image to canvas
        image = Image.open(f"samples/{sample}/spatial/tissue_hires_image.png")
        canvas.paste(image, (x*max_width, y*max_height))

        # keep the original spatial embedings in case you need them for latter
        # otherwise you can comment the next line, some image people may want
        # this information to go back to the "oroginal" (ndpi or tiff files)
        # histology images used as input for spaceranger
        adata.obsm['spatial_original'] = adata.obsm['spatial']

        # scale the embedings to match the tissue_hires_image
        scale_factor = adata.uns["spatial"][sample]['scalefactors']['tissue_hires_scalef']
        lib_mask = adata.obs['library_id']==sample
        adata.obsm['spatial'][lib_mask] = (adata.obsm['spatial'][lib_mask] * scale_factor).astype(np.float32)

        # offest embedings for this sample
        adata.obsm['spatial'][lib_mask,0] += x*max_width
        adata.obsm['spatial'][lib_mask,1] += y*max_height
        
        # show what's happening
        print(f"{sample} [{y}][{x}]; offset {sample} coords x+={x*max_height},y+={y*max_width}")
        x = x + 1
    y = y + 1

# save an image with all the merged libraries
canvas.save("visium_merged.png")


#### Sanity check: scatterplot for the spatial embedings

The plot will be upside down because we used top left for origin of coordinates instead of bottom left like the scatter.

In [ ]:
plt.scatter(adata.obsm["spatial"][:,0],adata.obsm["spatial"][:,1])


#### Save original metadata

In [ ]:
adata.uns['spatial_original_medatada'] = adata.uns['spatial']

### Craft dummy metadata for the merged image

In [ ]:
from matplotlib.pyplot import imread

# generate fake spatial annotation for cellxgene to use
spot_diameter = [adata.uns["spatial"][library_id]['scalefactors']['spot_diameter_fullres'] for library_id in adata.uns["spatial"].keys()]
fiducial_diameter = [adata.uns["spatial"][library_id]['scalefactors']['fiducial_diameter_fullres'] for library_id in adata.uns["spatial"].keys()]

uns_spatial = {
    'merged':{
        "images":{
            "hires": imread("visium_merged.png")
        },
        "scalefactors": {
            # scaleref is 1.0 because all coords have already been scaled
            'tissue_hires_scalef': 1.0,
            # average spot diamenters
            'spot_diameter_fullres': sum(spot_diameter)/len(spot_diameter),
            'fiducial_diameter_fullres': sum(fiducial_diameter)/len(fiducial_diameter)
        }
    }
}

In [ ]:
adata.uns['spatial'] = uns_spatial
# cellxgene needs `X_` prefixed obsm for plotting
adata.obsm['X_spatial'] = adata.obsm['spatial']

### Verify results with scanpy.pl.spatial

In [ ]:
sc.pl.spatial(adata, color='library_id', img_key='hires')

In [ ]:
adata.write('/path/for/save/visium_merged.h5ad', compression = 'gzip')